# Init values

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
SEED = 42
# dataset_folder = "datasets/financial"
# dataset_folder = "datasets/healthcare"
dataset_folder = "datasets/eligibility"


MAX_ITERATIONS = 1
TEMPERATURE = 0.9
LLM_NAME = "meta-llama/llama-3.1-8b-instruct"
# LLM_NAME = "meta-llama/llama-3.1-8b-instruct:free"


In [3]:
from dotenv import load_dotenv
from langchain_community.cache import SQLiteCache
from langchain.globals import set_llm_cache
from caia.utils import save_yaml_results
from caia.utils import ChatOpenRouter

import json
import uuid


# dataset_folder = "datasets/financial"
with open(f'{dataset_folder}/dataset_description.json', 'r') as f:
    dataset_description = json.load(f)

load_dotenv("env")
# set_llm_cache(SQLiteCache(database_path=".cache_langchain.db"))

llm_generator = ChatOpenRouter(model_name=LLM_NAME, cache=False,
                               temperature=TEMPERATURE) 

In [4]:
from rich import print

training_code = """
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

# load the old data
dataset_folder = "datasets/eligibility"
X_train_old = pd.read_csv(f"{dataset_folder}/X_train_old.csv")
X_test_old = pd.read_csv(f"{dataset_folder}/X_test_old.csv")
y_train_old = pd.read_csv(f"{dataset_folder}/y_train_old.csv").squeeze("columns")
y_test_old = pd.read_csv(f"{dataset_folder}/y_test_old.csv").squeeze("columns")

model_old = RandomForestClassifier(random_state=42)


model_old.fit(X_train_old, y_train_old)

# Test the model on the old test set
old_accuracy = model_old.score(X_test_old, y_test_old)

print(f'Model trained and evaluated on the old distribution: {old_accuracy}')
"""
print(training_code)

import pandas as pd
from sklearn.ensemble import RandomForestClassifier

# load the old data
dataset_folder = "datasets/eligibility"
X_train_old = pd.read_csv(f"{dataset_folder}/X_train_old.csv")
X_test_old = pd.read_csv(f"{dataset_folder}/X_test_old.csv")
y_train_old = pd.read_csv(f"{dataset_folder}/y_train_old.csv").squeeze("columns")
y_test_old = pd.read_csv(f"{dataset_folder}/y_test_old.csv").squeeze("columns")

model_old = RandomForestClassifier(random_state=42)


model_old.fit(X_train_old, y_train_old)

# Test the model on the old test set
old_accuracy = model_old.score(X_test_old, y_test_old)

print(f'Model trained and evaluated on the old distribution: {old_accuracy}')

In [5]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier


# load the reference data
X_train_old = pd.read_csv(f"{dataset_folder}/X_train_old.csv")
X_test_old = pd.read_csv(f"{dataset_folder}/X_test_old.csv")
y_train_old = pd.read_csv(f"{dataset_folder}/y_train_old.csv").squeeze("columns")
y_test_old = pd.read_csv(f"{dataset_folder}/y_test_old.csv").squeeze("columns")

print(f"X_train_old shape: {X_train_old.shape}")
print(f"X_test_old shape: {X_test_old.shape}")

model_old = RandomForestClassifier(random_state=SEED)
model_old.fit(X_train_old, y_train_old)

# Test the model on the initial test set
initial_accuracy = model_old.score(X_test_old, y_test_old)

print(f'Model trained and evaluated on the old distribution: {initial_accuracy}')

# Test the model on the drifted test set
X_test_new = pd.read_csv(f"{dataset_folder}/X_test_new.csv")
y_test_new = pd.read_csv(f"{dataset_folder}/y_test_new.csv").squeeze("columns")

print(f"X_test_new shape: {X_test_new.shape}")
drifted_accuracy = model_old.score(X_test_new, y_test_new)
print(f'Model evaluated on the new distribution: {drifted_accuracy}')

# calcualte the average accuracy
average_accuracy = (initial_accuracy + drifted_accuracy) / 2
print(f'Average accuracy on both distributions: {average_accuracy}')

X_train_old shape: (700, 5)

X_test_old shape: (300, 5)

Model trained and evaluated on the old distribution: 0.7833333333333333

X_test_new shape: (30, 5)

Model evaluated on the new distribution: 0.6333333333333333

Average accuracy on both distributions: 0.7083333333333333

In [6]:
metrics = {"model_old_score": {
            "on_new_data": drifted_accuracy,
            "on_old_data": initial_accuracy
        }
}
print(metrics)

{'model_old_score': {'on_new_data': 0.6333333333333333, 'on_old_data': 0.7833333333333333}}

# Baseline agent

In [7]:
from caia.benchmark.baseline import StandardGraph
standard_graph = StandardGraph(llm_generator, debug=False)

initial_state = {
    "model_code": training_code,
    "metrics":metrics,
    "max_iterations": MAX_ITERATIONS,
    "dataset_description": dataset_description
}

output = standard_graph.run(initial_state)

short_uuid = str(uuid.uuid4())[:8]
filename = f"results/baseline_temp_{TEMPERATURE}_max_iter_{MAX_ITERATIONS}_llm_{LLM_NAME.split('/')[1].split(':')[0]}_dataset_{dataset_folder.split('/')[-1]}_{short_uuid}.yaml"
save_yaml_results(output, filename)

Max iterations set to: 1

🚀 Starting Improved Baseline Model Improvement Process

Dataset: Eligibility Simulation Data

Features: 5 total, 2 numerical, 3 categorical

Executing Node: improve_code

...

Initial metrics: {'old_distribution': 0.7833333333333333, 'new_distribution': 0.6333333333333333}

Final metrics: {'old_distribution': 0.7833333333333333, 'new_distribution': 0.6333333333333333}

Improvement path: 0 entries

Total tokens used: 1703

Results saved to: results/baseline_temp_0.9_max_iter_1_llm_llama-3.1-8b-instruct_dataset_eligibility_617a0982.yaml

# ReAct agent

In [45]:
from caia.benchmark.react import ReactImprover

# Initialize the React improver with your LLM
react_graph = ReactImprover(llm_generator)

# Prepare initial state
initial_state = {
    "model_code": training_code,
    "metrics": metrics,
    "max_iterations": MAX_ITERATIONS,
    "dataset_description": dataset_description
}

# Run the agent
output = react_graph.run(initial_state)

short_uuid = str(uuid.uuid4())[:8]
filename = f"results/react_temp_{TEMPERATURE}_max_iter_{MAX_ITERATIONS}_llm_{LLM_NAME.split('/')[1].split(':')[0]}_dataset_{dataset_folder.split('/')[-1]}_{short_uuid}.yaml"
save_yaml_results(output, filename)

Max iterations set to: 1

🚀 Starting React-based Model Improvement Process

Dataset: Eligibility Simulation Data

Error handling: stopping after 3 consecutive failures

Features: 5 total, 2 numerical, 3 categorical

...

New Distribution: +0.0000

Distribution Gap: 0.1533 (changed by +0.0000)

Iteration Times:

Iteration 1: 9.66 seconds

Results saved to: results/react_temp_0.9_max_iter_1_llm_llama-3.1-8b-instruct_dataset_eligibility_679bbb05.yaml

# Plan and execute agent

In [9]:
from caia.benchmark.plan_and_execute import PlanAndExecuteGraph

# Initialize with max_failures parameter
plan_execute_graph = PlanAndExecuteGraph(
    llm_generator, 
    max_iterations=MAX_ITERATIONS,
    max_failures=3  # Will stop after 3 consecutive failures
)

# Prepare initial state
initial_state = {
    "model_code": training_code,
    "metrics": metrics,
    "dataset_description": dataset_description
}

# Run the agent
output = plan_execute_graph.run(initial_state)

# create a short version of uuid using python


short_uuid = str(uuid.uuid4())[:8]
filename = f"results/planexecute_temp_{TEMPERATURE}_max_iter_{MAX_ITERATIONS}_llm_{LLM_NAME.split('/')[1].split(':')[0]}_dataset_{dataset_folder.split('/')[-1]}_{short_uuid}.yaml"


save_yaml_results(output, filename)

🚀 Starting Plan-and-Execute Model Improvement Process

Dataset: Eligibility Simulation Data

Error handling: stopping after 3 consecutive failures

Features: 5 total, 2 numerical, 3 categorical

🧠 PLANNING STEP

...

Initial metrics: {'old_distribution': 0.7833333333333333, 'new_distribution': 0.6333333333333333}

Final metrics: {'old_distribution': 0.7833333333333333, 'new_distribution': 0.6333333333333333}

Improvement path: 0 entries

Total tokens used: 14131

Results saved to: 
results/planexecute_temp_0.9_max_iter_1_llm_llama-3.1-8b-instruct_dataset_eligibility_cd6b0e78.yaml

# Reflection agent

In [10]:
from caia.benchmark.reflection import ReflectionGraph


# Initialize with both max_iterations and max_failures parameters
reflection_graph = ReflectionGraph(
    llm_generator, 
    max_iterations=MAX_ITERATIONS,
    max_failures=3  # Will stop after 3 consecutive failures
)

# Prepare initial state
initial_state = {
    "model_code": training_code,
    "metrics": metrics,
    "dataset_description": dataset_description
}

# Run the agent
output = reflection_graph.run(initial_state)

short_uuid = str(uuid.uuid4())[:8]
filename = f"results/reflection_temp_{TEMPERATURE}_max_iter_{MAX_ITERATIONS}_llm_{LLM_NAME.split('/')[1].split(':')[0]}_dataset_{dataset_folder.split('/')[-1]}_{short_uuid}.yaml"
save_yaml_results(output, filename)

🚀 Starting Reflection-Based Model Improvement Process

Dataset: Eligibility Simulation Data

Error handling: stopping after 3 consecutive failures

Features: 5 total, 2 numerical, 3 categorical

🔍 GENERATING IMPROVED CODE

...

Final metrics: {'old_distribution': 0.8, 'new_distribution': 0.6666666666666666}

Improvement path: 1 entries

Reflections: 1

Total tokens used: 6216

Results saved to: 
results/reflection_temp_0.9_max_iter_1_llm_llama-3.1-8b-instruct_dataset_eligibility_4d884ce9.yaml

# ToT

In [11]:
from caia.benchmark.tot import TreeOfThoughtsGraph
import json



# Initialize with ToT-specific parameters
tot_graph = TreeOfThoughtsGraph(
    llm_generator, 
    max_iterations=MAX_ITERATIONS,
    beam_width=3,           # Number of candidates to keep after pruning
    num_candidates=3,       # Number of candidates to generate in each expansion
    threshold=0.9,          # Score threshold for accepting a solution
    max_depth=3,            # Maximum search depth in the tree
    max_failures=3          # Will stop after 3 consecutive failures
)

# Prepare initial state
initial_state = {
    "model_code": training_code,
    "metrics": metrics,
    "dataset_description": dataset_description
}

# Run the agent
output = tot_graph.run(initial_state)

short_uuid = str(uuid.uuid4())[:8]
filename = f"results/tot_temp_{TEMPERATURE}_max_iter_{MAX_ITERATIONS}_llm_{LLM_NAME.split('/')[1].split(':')[0]}_dataset_{dataset_folder.split('/')[-1]}_{short_uuid}.yaml"
save_yaml_results(output, filename)

🚀 Starting Tree of Thoughts Model Improvement Process

Parameters: max_iterations=1, beam_width=3, num_candidates=3

Error handling: stopping after 3 consecutive failures

Dataset: Eligibility Simulation Data

Features: 5 total, 2 numerical, 3 categorical

...

Final metrics: {'old_distribution': 0.8033333333333333, 'new_distribution': 0.6}

Improvement path: 1 entries

Total tokens used: 2755

Execution stats: 1 successes, 0 failures

Results saved to: results/tot_temp_0.9_max_iter_1_llm_llama-3.1-8b-instruct_dataset_eligibility_14bb6e39.yaml

# Self-Discovery

In [21]:
from caia.benchmark.self_discover import SelfDiscoverGraph

# Initialize with both max_iterations and max_failures
self_discovery_agent = SelfDiscoverGraph(
    llm_generator, 
    max_iterations=MAX_ITERATIONS,
    max_failures=4  # Will stop after 3 consecutive failures
)

# Prepare initial state
initial_state = {
    "model_code": training_code,
    "metrics": metrics,
    "dataset_description": dataset_description
}

# Run the agent
output = self_discovery_agent.run(initial_state)

short_uuid = str(uuid.uuid4())[:8]
filename = f"results/selfdiscovery_temp_{TEMPERATURE}_max_iter_{MAX_ITERATIONS}_llm_{LLM_NAME.split('/')[1].split(':')[0]}_dataset_{dataset_folder.split('/')[-1]}_{short_uuid}.yaml"
save_yaml_results(output, filename)


🚀 Starting Self-Discovery Model Improvement Process
Dataset: Eligibility Simulation Data
Error handling: stopping after 4 consecutive failures
Features: 5 total, 2 numerical, 3 categorical

🔍 SELECTING REASONING MODULES
Selected modules: 3
- 1. How could I simpl...
- 2. What are the key ...
- 3. How can I impleme...

Current Token Usage:
Prompt: 0
Completion: 0
Total: 0

🛠️ ADAPTING MODULES
Adapted modules: Here are the adapted modules to create a practical ML solution for the given problem:

**Module 1: Simplify the problem**

To simplify the problem, we can focus on the core challenge of predicting health outcomes based on the provided datasets. We can ignore the "old" and "new" distinction, and instead, aim to create a robust model that generalizes well across different data distributions.

**Module 2: Key techniques for improving an ML model on distribution shifts**

To address distribution shifts, we can apply the following techniques:

1. **Data preprocessing**: Use techniques l

Results saved to: 
results/selfdiscovery_temp_0.9_max_iter_1_llm_llama-3.1-8b-instruct_dataset_eligibility_f62a9945.yaml

# Improver

## Semantic memory

### Training code

In [30]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

# load the old data
# dataset_folder = "datasets/healthcare"
X_train_old = pd.read_csv(f"{dataset_folder}/X_train_old.csv")
X_test_old = pd.read_csv(f"{dataset_folder}/X_test_old.csv")
y_train_old = pd.read_csv(f"{dataset_folder}/y_train_old.csv").squeeze("columns")
y_test_old = pd.read_csv(f"{dataset_folder}/y_test_old.csv").squeeze("columns")

model_old = RandomForestClassifier(random_state=42)


model_old.fit(X_train_old, y_train_old)

# Test the model on the old test set
old_accuracy = model_old.score(X_test_old, y_test_old)

print(f'Model trained and evaluated on the old distribution: {old_accuracy}')

Model trained and evaluated on the old distribution: 0.7833333333333333

In [31]:
from caia.memory import WorkingMemory, EpisodicMemory, SemanticMemory
from caia.memory import Dataset


# tools = get_tools([calculate_trust_score])


# At the beginning, the agent has 1 entry in the semantic memory. 
# Here we put the path of each dataset file in the semantic memory.
dataset_old = Dataset(X_train=f"{dataset_folder}/X_train_old.csv",
                                     X_test=f"{dataset_folder}/X_test_old.csv",
                                     y_train=f"{dataset_folder}/y_train_old.csv",
                                     y_test=f"{dataset_folder}/y_test_old.csv",
                                     description=dataset_description)

model_code = """
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

# load the old data
dataset_folder = "datasets/eligibility"
X_train_old = pd.read_csv(f"{dataset_folder}/X_train_old.csv")
X_test_old = pd.read_csv(f"{dataset_folder}/X_test_old.csv")
y_train_old = pd.read_csv(f"{dataset_folder}/y_train_old.csv").squeeze("columns")
y_test_old = pd.read_csv(f"{dataset_folder}/y_test_old.csv").squeeze("columns")

model_old = RandomForestClassifier(random_state=42)


model_old.fit(X_train_old, y_train_old)

# Test the model on the old test set
old_accuracy = model_old.score(X_test_old, y_test_old)

print(f'Model trained and evaluated on the old distribution: {old_accuracy}')
"""

init_semantic_memory = SemanticMemory(dataset_old=dataset_old, 
                                        model_object=model_old, 
                                        model_code=model_code)
# semantic_memory
print(init_semantic_memory.model_code)

import pandas as pd
from sklearn.ensemble import RandomForestClassifier

# load the old data
dataset_folder = "datasets/eligibility"
X_train_old = pd.read_csv(f"{dataset_folder}/X_train_old.csv")
X_test_old = pd.read_csv(f"{dataset_folder}/X_test_old.csv")
y_train_old = pd.read_csv(f"{dataset_folder}/y_train_old.csv").squeeze("columns")
y_test_old = pd.read_csv(f"{dataset_folder}/y_test_old.csv").squeeze("columns")

model_old = RandomForestClassifier(random_state=42)


model_old.fit(X_train_old, y_train_old)

# Test the model on the old test set
old_accuracy = model_old.score(X_test_old, y_test_old)

print(f'Model trained and evaluated on the old distribution: {old_accuracy}')

## Episodic memory

In [32]:
from caia.memory import Dataset
from docarray import DocList

dataset_new = Dataset(X_train=f"{dataset_folder}/X_train_new.csv",
                        X_test=f"{dataset_folder}/X_test_new.csv",
                        y_train=f"{dataset_folder}/y_train_new.csv",
                        y_test=f"{dataset_folder}/y_test_new.csv",
                        description=dataset_description)


first_episodic_memory = EpisodicMemory(dataset_new=dataset_new,
                                        quick_insight={},
                                       deep_insight=None)
init_episodic_memory = DocList[EpisodicMemory]([first_episodic_memory])
init_episodic_memory[0]


📄 EpisodicMemory : c61f8bd ...
╭────────────────────────────┬─────────╮
│ Attribute                  │ Value   │
├────────────────────────────┼─────────┤
│ quick_insight: dict        │ {}      │
╰────────────────────────────┴─────────╯
└── 🔶 dataset_new: Dataset
    └── 📄 Dataset : 207381c ...
        ╭───────────────────┬───────────────────────────────────────────────────────────────────────╮
        │ Attribute         │ Value                                                                 │
        ├───────────────────┼───────────────────────────────────────────────────────────────────────┤
        │ X_train: str      │ datasets/eligibility/X_train_new.csv                                  │
        │ X_test: str       │ datasets/eligibility/X_test_new.csv                                   │
        │ y_train: str      │ datasets/eligibility/y_train_new.csv                                  │
        │ y_test: str       │ datasets/eligibility/y_test_new.csv                                   │
        │ description: dict │ {'NUM_SAMPLES': 1000, 'FEATURES': ['Age', 'Income' ... } (length: 11) │
        ╰───────────────────┴───────────────────────────────────────────────────────────────────────╯

## Fast graph

In [33]:
from caia.fast.fast_graph import FastGraph
from caia.utils import save_yaml_results

working_memory = WorkingMemory(
    episodic_memory=init_episodic_memory,
    semantic_memory=init_semantic_memory,
    threshold=0.05,
    generations_fast_graph={},
    generations_slow_graph={},
    improvement_history=[],
)

fast_graph = FastGraph(llm_generator, debug=False)
output_fast_graph = fast_graph.run(working_memory)

short_uuid = str(uuid.uuid4())[:8]
filename = f"results/fast_temp_{TEMPERATURE}_max_iter_{MAX_ITERATIONS}_llm_{LLM_NAME.split('/')[1].split(':')[0]}_dataset_{dataset_folder.split('/')[-1]}_{short_uuid}.yaml"
save_yaml_results(output_fast_graph, filename)

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                         Node: generate_retraining_code                                          │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

No slow graph insights available, using basic retraining approach

╭─────────────────────────────────────────────── new_training_code ───────────────────────────────────────────────╮
│ new_training_code: |                                                                                            │
│     import yaml                                                                                                 │
│     import pandas as pd                                                                                         │
│     from sklearn.ensemble import RandomForestClassifier                                                         │
│                                                                                                                 │
│     # Initialize metrics dictionaries                                                                           │
│     model_new_score = {                                                                                         │
│         'on_new_data': 0.0,                                                                                     │
│         'on_old_data': 0.0                                                                                      │
│     }                                                                                                           │
│     model_old_score = {                                                                                         │
│         'on_new_data': 0.0,                                                                                     │
│         'on_old_data': 0.0                                                                                      │
│     }                                                                                                           │
│                                                                                                                 │
│     # load the old data                                                                                         │
│     dataset_folder = "datasets/eligibility"                                                                     │
│     X_train_old = pd.read_csv(f"{dataset_folder}/X_train_old.csv")                                              │
│     X_test_old = pd.read_csv(f"{dataset_folder}/X_test_old.csv")                                                │
│     y_train_old = pd.read_csv(f"{dataset_folder}/y_train_old.csv").squeeze("columns")                           │
│     y_test_old = pd.read_csv(f"{dataset_folder}/y_test_old.csv").squeeze("columns")                             │
│                                                                                                                 │
│     # Train and evaluate old model                                                                              │
│     model_old = RandomForestClassifier(random_state=42)                                                         │
│     model_old.fit(X_train_old, y_train_old)                                                                     │
│                                                                                                                 │
│     # Test old model on old test set                                                                            │
│     old_score_old = model_old.score(X_test_old, y_test_old)                                                     │
│     print(f'Old model trained and evaluated on the old distribution: {old_score_old}')                          │
│     model_old_score['on_old_data'] = float(old_score_old)                                                       │
│                                                                                                                 │
│     # Test old model on new test set                                                                            │
│     X_test_new = pd.read_csv(f"{dataset_folder}/X_test_new.csv")                                                │
│     y_test_new = pd.read_csv(f"{dataset_folder}/y_test

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                          Node: execute_retraining_code                                          │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


>>>>>>>> EXECUTING CODE BLOCK (inferred language is python)...


...

╭─────────────────────────────────────────────── execution_success ───────────────────────────────────────────────╮
│ True                                                                                                            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── extracted_metrics ───────────────────────────────────────────────╮
│ {'model_old_score': {'on_old_data': 0.7833333333333333, 'on_new_data': 0.6333333333333333}, 'model_new_score':  │
│ {'on_old_data': 0.7866666666666666, 'on_new_data': 0.6333333333333333}}                                         │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── iteration_count ────────────────────────────────────────────────╮
│ 1                                                                                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────── Latest Improvement ───────────────────────────────────────────────╮
│ Outcome: success                                                                                                │
│ Improvements:                                                                                                   │
│   New Distribution: 0.0000                                                                                      │
│   Old Distribution: 0.0033                                                                                      │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Results saved to: results/fast_temp_0.9_max_iter_1_llm_llama-3.1-8b-instruct_dataset_eligibility_b11be41d.yaml

## Slow graph

In [40]:
from caia.slow.slow_graph import SlowGraph
from caia.utils import save_yaml_results

slow_graph = SlowGraph(llm_generator, debug=False)
working_memory["max_iterations"] = MAX_ITERATIONS
working_memory["max_failures"] = 5
output_slow_graph = slow_graph.run(working_memory)


short_uuid = str(uuid.uuid4())[:8]
filename = f"results/slow_temp_{TEMPERATURE}_max_iter_{MAX_ITERATIONS}_llm_{LLM_NAME.split('/')[1].split(':')[0]}_dataset_{dataset_folder.split('/')[-1]}_{short_uuid}.yaml"
save_yaml_results(output_slow_graph, filename)

Max iterations set to: 1

Max consecutive failures set to: 5

==================== STARTING ITERATION 1/1 ====================

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                         Node: check_fast_graph_results                                          │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Detected Fast Graph Results from generations_fast_graph: --------------------------------------------------

...

╭────────────────────────────────────────────── Latest Improvement ───────────────────────────────────────────────╮
│ Strategy: hyperparameter_tuning                                                                                 │
│ Outcome: success                                                                                                │
│ Improvements:                                                                                                   │
│   New Distribution: 0.0333                                                                                      │
│   Old Distribution: -0.0200                                                                                     │
│ Evaluation: reject                                                                                              │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── Strategy Progress ───────────────────────────────────────────────╮
│   [○] model_selection                                                                                           │
│ → [✓] hyperparameter_tuning                                                                                     │
│   [○] ensemble_method                                                                                           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Iteration 1 time: 166.91 seconds

Keeping Slow Graph results: Slow Graph total=1.4333 >= Fast Graph total=1.4200

Results saved to: results/slow_temp_0.9_max_iter_1_llm_llama-3.1-8b-instruct_dataset_eligibility_6f0d2dee.yaml

## Fast graph again

In [42]:
from caia.fast.fast_graph import FastGraph
from caia.utils import save_yaml_results

working_memory = WorkingMemory(
    episodic_memory=init_episodic_memory,
    semantic_memory=init_semantic_memory,
    threshold=0.05,
    generations_fast_graph={},
    generations_slow_graph=output_slow_graph,
    improvement_history=[],
)

fast_graph = FastGraph(llm_generator, debug=False)
output_fast_graph = fast_graph.run(working_memory)

short_uuid = str(uuid.uuid4())[:8]
print(short_uuid)
filename = f"results/improver_temp_{TEMPERATURE}_max_iter_{MAX_ITERATIONS}_llm_{LLM_NAME.split('/')[1].split(':')[0]}_dataset_{dataset_folder.split('/')[-1]}_{short_uuid}.yaml"
save_yaml_results(output_fast_graph, filename)

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                         Node: generate_retraining_code                                          │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Using insights from slow graph to enhance retraining code generation

╭──────────────────────────────────────────── has_slow_graph_insights ────────────────────────────────────────────╮
│ True                                                                                                            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── new_training_code ───────────────────────────────────────────────╮
│ new_training_code: |                                                                                            │
│   import yaml                                                                                                   │
│   import pandas as pd                                                                                           │
│   from sklearn.ensemble import RandomForestClassifier                                                           │
│   from sklearn.metrics import accuracy_score                                                                    │
│                                                                                                                 │
│   # Initialize metrics dictionaries                                                                             │
│   model_new_score = {                                                                                           │
│       'on_new_data': 0.0,                                                                                       │
│       'on_old_data': 0.0                                                                                        │
│   }                                                                                                             │
│                                                                                                                 │
│   model_old_score = {                                                                                           │
│       'on_new_data': 0.0,                                                                                       │
│       'on_old_data': 0.0                                                                                        │
│   }                                                                                                             │
│                                                                                                                 │
│   try:                                                                                                          │
│     # Load data from specified folder                                                                           │
│     dataset_folder = "datasets/eligibility"                                                                     │
│     X_train_old = pd.read_csv(f"{dataset_folder}/X_train_old.csv")                                              │
│     X_test_old = pd.read_csv(f"{dataset_folder}/X_test_old.csv")                                                │
│     y_train_old = pd.read_csv(f"{dataset_folder}/y_train_old.csv").squeeze("columns")                           │
│     y_test_old = pd.read_csv(f"{dataset_folder}/y_test_old.csv").squeeze("columns")                             │
│                                                                                                                 │
│     # Load new data                                                                                             │
│     X_train_new = pd.read_csv(f"{dataset_folder}/X_train_new.csv")                                              │
│     y_train_new = pd.read_csv(f"{dataset_folder}/y_train_new.csv").squeeze("columns")                           │
│     X_test_new = pd.read_csv(f"{dataset_folder}/X_test_new.csv")                                                │
│     y_test_new = pd.read_csv(f"{dataset_folder}/y_test_new.csv").squeeze("columns")                             │
│                                                                                                                 │
│     # Define metrics dictionary                                                                                 │
│     model_large_score = {                                                                                       │
│         'on_new_data': 0.0,                           

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                          Node: execute_retraining_code                                          │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭──────────────────────────────────────────────── iteration_count ────────────────────────────────────────────────╮
│ 1                                                                                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────── Latest Improvement ───────────────────────────────────────────────╮
│ Outcome: success                                                                                                │
│ Improvements:                                                                                                   │
│   New Distribution: 0.0000                                                                                      │
│   Old Distribution: 0.0033                                                                                      │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Keeping current Fast Graph results: Current total=1.4400 >= Best baseline total=1.4400

3b18ff66

Results saved to: results/improver_temp_0.9_max_iter_1_llm_llama-3.1-8b-instruct_dataset_eligibility_3b18ff66.yaml